In [ ]:
#Yassin TAlssis
%matplotlib inline
from PIL import Image
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import timeit

device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

In [ ]:
# load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# summarize loaded dataset
print(f'Train: X= {x_train.shape}, y= {y_train.shape}')
print(f'Test: X={x_test.shape}, y={y_test.shape}')
print("Some of the Images in the Dataset are displayed below")

figure = plt.figure(figsize=(12,12))
for i in range(12, 21):
# define subplot
    plt.subplot(3,3,i-11)
# plot raw pixel data
    plt.xlabel(y_train[i])
    plt.imshow(x_train[i])
# show the figure
plt.show()

In [ ]:
print(x_train.dtype, x_test.dtype)
# Normalizing data 
# As the Data is of type uint8 we will convert it to Float
x_train = x_train.astype(float)
x_test = x_test.astype(float)
x_train = x_train/255.0
x_test = x_test/255.0

In [ ]:
def visualize_results(history):
    figure = plt.figure(figsize=(15, 10))
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='orange', label='test')

# ResNet34

In [ ]:
def convolutional_block(x, filter):
    
    # copy tensor to variable called x_skip
    x_skip = x
    
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    
    return x

def identity_block(x, filter):
    
    # copy tensor to variable called x_skip
    x_skip = x
    
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    
    return x
def ResNet34(shape = (32, 32, 3), classes = 10):
  
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
          
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

model = ResNet34()
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
model.summary()

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs = 15)
visualize_results(history)

# VGG

In [ ]:
# VGG Based Architechture (No residual Blocks)
"""
Architecture:

Block Architecture:
Conv2D
Batch Normalization
Canv2D
Batch Normalization
MaxPool
Dropout

Stack Blocks
Fully Connected Dense Layers
Output Layer
"""
def model_training():
    class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') >= 0.96):
                print("The model has reached 96% train accuracy !")
                self.model.stop_training = True

    callback = MyCallback()

    model = tf.keras.models.Sequential([
      # BLOCK-1
          tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape=(32,32,3), padding = 'same'),
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape=(32,32,3), padding = 'same'),
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.MaxPool2D((2,2)),
          tf.keras.layers.Dropout(0.2),
      # BLOCK-2
          tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', input_shape=(32,32,3), padding = 'same'),
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', input_shape=(32,32,3), padding = 'same'),
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.MaxPool2D((2,2)), 
          tf.keras.layers.Dropout(0.3),
      # BLOCK-3    
          tf.keras.layers.Conv2D(128, (3,3), activation = 'relu', input_shape=(32,32,3), padding = 'same'),
          tf.keras.layers.BatchNormalization(), 
          tf.keras.layers.Conv2D(128, (3,3), activation = 'relu', input_shape=(32,32,3), padding = 'same'),
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.MaxPool2D((2,2)),
          tf.keras.layers.Dropout(0.4),
      # BLOCK-4
          tf.keras.layers.Conv2D(256, (3,3), activation = 'relu', input_shape=(32,32,3), padding = 'same'),
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.Conv2D(256, (3,3), activation = 'relu', input_shape=(32,32,3), padding = 'same'),
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.MaxPool2D((2,2)),
          tf.keras.layers.Dropout(0.4),
      # End BLOCK-(FC Dense with BN and Activation = RELU)
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(512, activation = 'relu'),
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.Dropout(0.5),
          tf.keras.layers.Dense(256, activation = 'relu'),
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.Dropout(0.7),
          tf.keras.layers.Dense(128, activation = 'relu'),
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.Dropout(0.7),
          tf.keras.layers.Dense(10, activation='softmax')
  ])

    model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

    print(model.summary())

    return model, callback

In [ ]:
# Data Augmentation can also be applied to further improve the accuracy 
"""
datagen = tf.keras.preprocessing.images.ImageDataGeterator(rotation_range=0,
    width_shift_range=1.0,
    height_shift_range=1.0,
    horzontal_flip = True)
aug_data = datagen.flow(x_train, x_test)
history = model.fit(aug_data, validation_data=(x_test, y_test), epochs = 150, callbacks=[callback])
visualize_results(history)
"""
# But due to Limited Resources Training the Model with Augmented Data was not possible 
# Platforms -(Google Colab and Kaggle Kernel (GPU Acceleration))
model1, callback = model_training()
history = model1.fit(x_train, y_train, validation_data=(x_test, y_test), epochs = 150, callbacks=[callback])
visualize_results(history)

In [ ]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model1.layers[1:]]

visualization_model = tf.keras.models.Model(inputs = model1.input, outputs = successive_outputs)


x = img_to_array(x_train[19])  # Numpy array with shape (32, 32, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 32, 32, 3)

x /= 255

plt.imshow(x_train[19])
pred = list(model2.predict(x)[0])
print(pred)
print(max(pred))
classes = pred.index(max(pred))
print("Prediction :", classes)

successive_feature_maps = visualization_model.predict(x)

layer_names = [layer.name for layer in model1.layers[1:]]

for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  # print(feature_map.shape)
  if len(feature_map.shape) == 4:
    # for the conv / maxpool layers, not the fully-connected layers
    n_features = feature_map.shape[-1]  # number of features in feature map
    # The feature map has shape (1, size, size, n_features)
    size = feature_map.shape[1]
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
        # Postprocess the feature to make it visually palatable
        x = feature_map[0, :, :, i]
        x -= x.mean()
        x /= x.std()
        x *= 64
        x += 128
        x = np.clip(x, 0, 255).astype('uint8')
        # We'll tile each filter into this big horizontal grid
        display_grid[:, i * size : (i + 1) * size] = x
    # Display the grid
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [ ]:
# In reference to SOTA Accuracy (Transfer Learning with ResNet50)
# Transfer Learning Approach using Resnet 50 Pretrained on Imagenet

def resnet():
        base_model = tf.keras.applications.ResNet50V2(
        include_top=False,
        weights='imagenet',
        input_shape=(32, 32, 3)
        )

        headModel = base_model.output
        headModel = tf.keras.layers.GlobalAveragePooling2D()(headModel)
        headModel = tf.keras.layers.Flatten()(headModel)
        headModel = tf.keras.layers.Dense(512, activation="relu")(headModel)
        headModel = tf.keras.layers.Dropout(0.7)(headModel)
        headModel = tf.keras.layers.Dense(10, activation="softmax")(headModel)

        model = tf.keras.models.Model(inputs = base_model.input, outputs = headModel)


        model.compile(optimizer = tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
        print(model.summary())
        return model

In [ ]:
train_callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5,
        patience=3, verbose=1
    )
]
model2 = resnet()
history = model2.fit(x_train, y_train, steps_per_epoch = 781, 
                     validation_data=(x_test, y_test), epochs = 100, callbacks=train_callbacks)
visualize_results(history)